# Continued Pre-Training with SmolLM2 on Turkish Language

In [ ]:
%%capture
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

from torch import __version__ as torch_version
from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(torch_version) < V("2.4.0") else "xformers"

!pip install -q --no-deps {xformers} trl peft accelerate bitsandbytes datasets wandb huggingface_hub sentencepiece

In [ ]:
import os, random, numpy as np, torch
from datasets import Dataset
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TrainingArguments
from trl import SFTTrainer

# reproducibility
random.seed(42); np.random.seed(42); torch.manual_seed(42)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Device: cuda


# Hugging Face & W&B auth

In [ ]:
from getpass import getpass
from huggingface_hub import login
import wandb

hf_token = getpass("🔑 Enter your Hugging Face token (or press Enter to skip): ").strip()
if hf_token:
    login(hf_token)
    hf_logged_in = True
    print("✅ HF login successful.")
else:
    hf_logged_in = False
    print("ℹ️ Skipping HF login.")

wb_token = getpass("🔑 Enter your Weights & Biases token (or press Enter to skip): ").strip()
if wb_token:
    wandb.login(key=wb_token)
    run = wandb.init(project="CPT_SmolLM2_NewLanguage_TR", job_type="training", anonymous="allow")
    os.environ["WANDB_DISABLED"] = "false"
    report_to = "wandb"
    print("✅ W&B logging enabled.")
else:
    os.environ["WANDB_DISABLED"] = "true"
    report_to = "none"
    print("ℹ️ W&B disabled.")

🔑 Enter your Hugging Face token (or press Enter to skip): ··········
✅ HF login successful.
🔑 Enter your Weights & Biases token (or press Enter to skip): ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aditya_rajpurohit (aditya_rajpurohit-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


✅ W&B logging enabled.


# Load base Language Model for Continued Pre-Training

In [ ]:
model_name = "unsloth/smollm2-135m"
max_seq_length = 512
dtype = None

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name       = model_name,
    max_seq_length   = max_seq_length,
    dtype            = dtype,
    load_in_4bit     = False,
    full_finetuning  = True,
    token            = hf_token if hf_logged_in else None,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable()

FastLanguageModel.for_training(model)
model.to(device)
print("✅ Model ready for CPT:", model_name)

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

✅ Model ready for CPT: unsloth/smollm2-135m


# Tiny Turkish monolingual corpus

In [ ]:
texts = [
    "Bu, dili öğrenmek için yazılmış kısa bir Türkçe cümledir.",
    "Yapay zeka, günlük hayatımızdaki pek çok sorunu çözmemize yardımcı oluyor.",
    "Her gün biraz pratik yaparsanız, dil becerileriniz yavaş yavaş gelişir.",
    "Veri bilimi ve makine öğrenimi günümüz dünyasında önemli beceriler haline geldi.",
    "Sınavdan önce sakin kalmak ve iyi notları tekrar etmek çok önemlidir.",
    "Yeni bir dil öğrenirken düzenli tekrar ve sabır çok değerlidir.",
    "Hata yapmaktan korkmayın; hatalar çoğu zaman en iyi öğretmendir.",
    "Basit cümlelerle başlayıp zamanla daha karmaşık yapılara geçin.",
    "Günlük yazı yazmak ve kısa metinler okumak akıcılığı artırır.",
    "Dinleme, konuşma, okuma ve yazma birlikte geliştiğinde güçlü olur.",
]

texts = texts * 200

cpt_dataset = Dataset.from_dict({"text": texts})
print("Samples:", len(cpt_dataset))
print("Preview:", cpt_dataset[0]["text"])

Samples: 2000
Preview: Bu, dili öğrenmek için yazılmış kısa bir Türkçe cümledir.


# SFTTrainer tokenizes

In [ ]:
def passthrough(examples):
    return {"text": examples["text"]}

cpt_text_ds = cpt_dataset.map(passthrough, batched=True)
print("Columns:", cpt_text_ds.column_names)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Columns: ['text']


# Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir                  = "smollm2-135m-cpt-tr",
    num_train_epochs            = 1,
    max_steps                   = 50,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 1,
    learning_rate               = 2e-4,
    warmup_steps                = 30,
    logging_steps               = 10,
    save_strategy               = "no",
    fp16                        = (torch.cuda.is_available() and not torch.cuda.is_bf16_supported()),
    bf16                        = (torch.cuda.is_available() and torch.cuda.is_bf16_supported()),
    report_to                   = report_to,
)
print("✅ Training args ready.")

✅ Training args ready.


# SFTTrainer

In [ ]:
cpt_trainer = SFTTrainer(
    model              = model,
    tokenizer          = tokenizer,
    train_dataset      = cpt_text_ds,
    dataset_text_field = "text",
    max_seq_length     = max_seq_length,
    packing            = True,
    args               = training_args,
)
print("✅ Trainer ready.")

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Trainer ready.


# Train the Model

In [ ]:
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} | VRAM: {round(gpu.total_memory/1e9, 2)} GB")

train_out = cpt_trainer.train()

mins = round(train_out.metrics.get("train_runtime", 0)/60, 2)
peak = f"{round(torch.cuda.max_memory_reserved()/1e9,3)} GB" if torch.cuda.is_available() else "CPU"
print(f"⏱ Runtime: {mins} min | 💾 Peak reserved GPU: {peak}")

The model is already on multiple devices. Skipping the move to device specified in `args`.


GPU: Tesla T4 | VRAM: 15.83 GB


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 134,515,584 of 134,515,584 (100.00% trained)


Step,Training Loss
10,3.164900
20,1.260900
30,0.418900
40,0.175700
50,0.070200


⏱ Runtime: 1.09 min | 💾 Peak reserved GPU: 2.376 GB


# Inference in Turkish

In [ ]:
FastLanguageModel.for_inference(model)
inference_dtype = torch.bfloat16 if is_bfloat16_supported() else (torch.float16 if torch.cuda.is_available() else torch.float32)
model = model.to(device=device, dtype=inference_dtype)

def gen_tr(prompt, max_new_tokens=100, temperature=0.8, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            **inputs,
            max_new_tokens = max_new_tokens,
            do_sample      = True,
            temperature    = temperature,
            top_p          = top_p,
            use_cache      = True,
        )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# test prompts in Turkish
gen_tr("Yapay zekanın eğitim alanındaki faydalarını iki cümlede özetleyin.")
gen_tr("Sınavlara çalışırken öğrencilerin dikkat etmesi gereken üç önemli noktayı yazın.")

Yapay zekanın eğitim alanındaki faydalarını iki cümlede özetleyin.

İlkrünlük önce sakin kalmak ve kalmak çok sorunu çok sorunu önemlidir.

İlk önemlidir. Çoğu zaman en iyi önemlidir. Çoğu zaman en önemlidir. Çoğu zaman en önemlidir. Çoğ
Sınavlara çalışırken öğrencilerin dikkat etmesi gereken üç önemli noktayı yazın.

İlk önemlidirken düzenli tekrar ve kalmak çoğu zamanla daha karmaşık yapılara geçin.

İlk önemlidirken düzenli tekrar ve kalmak çoğu zamanla daha karmaşık yapılara geçin.

Önemlidirken
